# Setup

Using a graph library really simplifies the problem. Sure, it takes ~4 seconds to build the graph with ~20500 edges, but once it's done it's trivial to find all shortest paths.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

with open('input.txt') as f:
    lines = f.readlines()
    data = [[c for c in line.strip()] for line in lines]

# 0: right, 1: down, 2: left, 3: up
offsets = [(0, 1), (1, 0), (0, -1), (-1, 0)]

board = np.array(data)
free_spaces: list[list[int]] = np.argwhere(board == '.').tolist()

start_point = tuple(np.argwhere(board == 'S')[0].tolist())
end_point = tuple(np.argwhere(board == 'E')[0].tolist())
free_spaces.append(start_point)
free_spaces.append(end_point)

def show_path(board, path):
    dir_chars = ['―', '|']
    used_board = board.copy()
    for i, node in enumerate(path):
        y, x = node[:2]
        if len(node) == 2:
            used_board[y, x] = 'E'
        elif i == 0:
            used_board[y, x] = 'S'
        else:
            d = node[2] % 2
            used_board[y, x] = dir_chars[d]
            
    used_board[path[0][0], path[0][1]] = 'S'
        
    print('\n'.join([''.join(line) for line in used_board.tolist()]))

graph = nx.Graph()

for y, x in free_spaces:
    graph.add_edge((y, x, 0), (y, x, 1), weight=1000)
    
    for dir_num, (dy, dx) in enumerate(offsets):
        dir_num %= 2
        offset_point = [y + dy, x + dx]
        if offset_point in free_spaces:
            # print(f'Adding edge from {y, x, dir_num} to {offset_point[0], offset_point[1], dir_num}')
            graph.add_edge((y, x, dir_num), (y + dy, x + dx, dir_num), weight=1)
            
for i in range(2):
    graph.add_edge(end_point, (*end_point, i), weight=0)

# Part 1

In [2]:
start_point = (*start_point, 0)
p = nx.shortest_path(graph, start_point, end_point, weight='weight')
print(nx.path_weight(graph, p, weight='weight'))

show_path(board, p)

99460
#############################################################################################################################################
#.......................#.......#.............#...........#.....#...#――――――|#――――――――|#――――――――――――――――――――――――――――――――|..#.......#........E#
#.#.#######.###.#########.#.###.###.#.#.#####.#######.###.###.#.#.###|#####|#|#####.#|#|#####.###.#.###.#####.#.#######|###.#####.#.#######|#
#.#.#....――――――――――――――――――――――――――|..#.....#...#...#...#.....#...#――|....#――|#...#.#|#|#...#.....#...#.#...#.#...#...#――|#...#.#...#.....#|#
#.###.#.#|#######.#########.#.###.#|###.###.###.#.#.###.#.#######.#|###########.#.#.#|#|#.#.#####.###.###.#.#.###.#.#.#.#|#.#.#.#####.#####|#
#.....#.#|#.......#...#...#.#.#...#――|#...#...#...#.....#.........#|..#...#...#.#...#|#|..#.#.......#.....#.#.....#.#.#.#――|....#.....#...#|#
#.#####.#|#.#######.#.###.#.#.#.#####|#####.#.###########.#########|###.#.#.#.#.#.###|#|###.#.###.#.#######.#.#####.#.#####|###.###.#.#.#.#|#


# Part 2

In [3]:
import itertools
paths = nx.all_shortest_paths(graph, start_point, end_point, weight='weight')
short_path_nodes = set()
for node in itertools.chain(*paths):
    short_path_nodes.add(node[:2])
    
len(short_path_nodes)

500